In [1]:
%reload_ext autoreload
%autoreload 2

from core.experiments import RunFactory

In [2]:
wandb_user = 'sisaman'
wandb_project = 'GAP-ATTACK'

run_factory = RunFactory(
    main_file='attack.py', 
    entity=wandb_user,
    project=wandb_project, 
    check_existing=True
)

# Experiments

### Hyper-parameters

In [3]:
datasets = ['facebook', 'reddit', 'amazon']
batch_size = 256


gap_methods  = ['gap-inf', 'gap-ndp']
sage_methods = ['sage-inf', 'sage-ndp']
mlp_methods  = ['mlp', 'mlp-dp']
ndp_methods  = ['gap-ndp', 'sage-ndp', 'mlp-dp']
all_methods  = gap_methods + sage_methods + mlp_methods

hparams = {dataset: {method: {} for method in all_methods} for dataset in datasets}

for dataset in datasets:
    # For GAP methods
    for method in gap_methods:
        hparams[dataset][method]['target_encoder_layers'] = 2
        hparams[dataset][method]['target_base_layers'] = 1
        hparams[dataset][method]['target_head_layers'] = 1
        hparams[dataset][method]['target_combine'] = 'cat'
        hparams[dataset][method]['target_hops'] = 2

    # For SAGE methods
    for method in sage_methods:
        hparams[dataset][method]['target_base_layers'] = 2
        hparams[dataset][method]['target_head_layers'] = 1
        if method != 'sage-ndp':
            hparams[dataset][method]['target_mp_layers'] = 2
        
    # For MLP methods
    for method in mlp_methods:
        hparams[dataset][method]['target_num_layers'] = 3

    # For GAP-NDP and SAGE-NDP
    for method in ['gap-ndp', 'sage-ndp']:
        hparams[dataset][method]['target_max_degree'] = 100

    # For all methods
    for method in all_methods:
        hparams[dataset][method]['target_hidden_dim'] = 64
        hparams[dataset][method]['target_activation'] = 'selu'
        hparams[dataset][method]['target_optimizer'] = 'adam'
        hparams[dataset][method]['target_learning_rate'] = 0.01
        
        if method in ndp_methods:
            hparams[dataset][method]['target_max_grad_norm'] = 1
            hparams[dataset][method]['target_epochs'] = 10
            hparams[dataset][method]['target_batch_size'] = 256

        else:
            hparams[dataset][method]['target_batch_norm'] = True
            hparams[dataset][method]['target_epochs'] = 100
            hparams[dataset][method]['target_batch_size'] = 'full'

        if method != 'sage-ndp':
            hparams[dataset][method]['target_val_interval'] = 0

        if method in gap_methods:
            hparams[dataset][method]['target_encoder_epochs'] = hparams[dataset][method]['target_epochs']

        hparams[dataset][method]['num_nodes_per_class'] = 1000
        hparams[dataset][method]['attack_hidden_dim'] = 64
        hparams[dataset][method]['attack_num_layers'] = 3
        hparams[dataset][method]['attack_activation'] = 'selu'
        hparams[dataset][method]['attack_batch_norm'] = True
        hparams[dataset][method]['attack_batch_size'] = 'full'
        hparams[dataset][method]['attack_epochs'] = 100
        hparams[dataset][method]['attack_optimizer'] = 'adam'
        hparams[dataset][method]['attack_learning_rate'] = 0.01
        hparams[dataset][method]['attack_val_interval'] = 1
        hparams[dataset][method]['repeats'] = 10


### Attack Experiments

In [4]:
for dataset in datasets:
    for method in all_methods:
        params = {}
        if method in ndp_methods:
            params['target_epsilon'] = [1, 2, 4, 8, 16]
        
        run_factory.register(
            method, 
            'nmi',
            dataset=dataset,
            **params, 
            **hparams[dataset][method]
        )

run_factory.save(path='jobs/gap-attack.sh')

Saved 54 runs to jobs/gap-attack.sh
